In [58]:
import gzip
import csv

phedex_columns={0: 'date',
                1: 'site',
                2: 'datset',
                3: 'size',
                4: 'orig_date',
                5: 'group'}
phedex_info={}
n=0

with gzip.open('phedex.out.gz') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        n=n+1
        key=(row[2],row[4],row[1])
        if 'null' in row[5]:
            row[5]=-99
            #print row
        if 'T3_' in row[1]: continue
        phedex_info[key]=(int(row[0]),row[1],int(row[3]),row[4],int(row[5]))
        if n<10: print row[2],phedex_info[key]
        
     
print("Done reading phedex info")
print phedex_columns
print len(phedex_info)

/W2JetsToLNu_LHEWpT_0-50_TuneCP5_13TeV-amcnloFXFX-pythia8/RunIIFall17DRPremix-PU2017_94X_mc2017_realistic_v11-v1/AODSIM (20181012, 'T1_RU_JINR_Disk', 44362131640, '20180801', 42)
/DoubleEG/Run2016F-17Jul2018-v1/MINIAOD (20181012, 'T2_US_MIT', 432248952686, '20180728', 42)
/TTGamma_SingleLeptFromTbar_TuneCUETP8M1_14TeV-madgraph-pythia8/PhaseIITDRFall17MiniAOD-PU200_93X_upgrade2023_realistic_v2-v2/MINIAODSIM (20181012, 'T1_US_FNAL_Disk', 154373888042, '20180106', 42)
/SingleMuon/Run2018C-MuAlCalIsolatedMu-PromptReco-v3/ALCARECO (20181012, 'T2_CH_CERN', 2417829152208, '20180822', 102)
/ChargedHiggs_HplusTB_HplusToTauNu_IntermediateMassNoNeutral_M-175_13TeV_amcatnlo_pythia8/RunIISummer16MiniAODv2-PUMoriond17_80X_mcRun2_asymptotic_2016_TrancheIV_v6-v1/MINIAODSIM (20181012, 'T1_US_FNAL_Disk', 105468730612, '20171229', 42)
/ST_t-channel_top_4f_InclusiveDecays_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIIFall17DRPremix-PU2017_94X_mc2017_realistic_v11-v1/AODSIM (20181012, 'T2_FR_GRIF_IRFU', 4829

In [49]:
import gzip
import numpy
myJInput=''
for line in gzip.open('popDaily.txt.gz'):
    myJInput += line

import json    
myJ=json.loads(myJInput)

pop_datasets={}
dates=myJ.keys()

minD=None
maxD=None
i=0
for i,d in enumerate(dates):
    npD=numpy.datetime64(d)
    if minD==None or minD>npD:
        minD=npD
    if maxD==None or maxD<npD:
        maxD=npD
        
#    if i%100==0:
#        print i,d
    records = myJ[d]['DATA']
    for record in records:
        dataset=record['COLLNAME']
        ending=dataset.split('/')[-1]
        #if 'AOD' not in ending: continue
        nacc=record["NACC"]
        if dataset not in pop_datasets:
            pop_datasets[dataset]={}
        pop_datasets[dataset][npD]=nacc
        
print "number of datasets in popularity info",len(datasets)
print "first date",minD,"last date",maxD
myJ=None

number of datasets in popularity info 500326
first date 2016-01-01 last date 2018-10-11


In [65]:
datasets = phedex_info.keys()
import numpy
import datetime
now_date = datetime.datetime.now()
np_now_date = numpy.datetime64(now_date)

sorted_by_size = reversed(sorted(phedex_info.items(), key=lambda kv: kv[1][2]))

nMax=-1
n=0

fOut=open('popOutput.txt','w')

for key,val in sorted_by_size:
    if n==nMax:continue
    n+=1
    dataset=key[0]    
    if 'RelVal' in dataset: continue
    #if 'MINIAOD' in dataset: continue
    if int(key[1])>20180801: continue
        
    pop_dataset = pop_datasets.get(dataset,None)
    if pop_dataset is None:
        if n<1000:
            print 'Never read',key,val[4],int(100*val[2]/1024./1024./1024./1024.)*0.01
        fOut.write('Never '+str(key)+' '+str(val[4])+' '+str(int(100*val[2]/1024./1024./1024./1024.)*0.01)+'\n')
    else:
        t=numpy.array(pop_dataset.keys())
        t.sort()
        lastRead= int((np_now_date - t[-1]) / numpy.timedelta64(1,'D'))
        if lastRead>100:
            if n<1000:
                print lastRead, key, val[4], int(100*val[2]/1024./1024./1024./1024.)*0.01
            fOut.write(str(lastRead)+' '+str(key)+' '+str(val[4])+' '+str(int(100*val[2]/1024./1024./1024./1024.)*0.01)+'\n')
fOut.close()
    


Never read ('/Neutrino_E-10_gun/RunIISpring15PrePremix-PUMoriond17_80X_mcRun2_asymptotic_2016_TrancheIV_v2-v2/GEN-SIM-DIGI-RAW', '20161028', 'T2_CH_CERN') 18 412.89
208 ('/MET/Run2017F-v1/RAW', '20180331', 'T1_RU_JINR_Disk') 42 160.67
188 ('/SingleMuon/Run2017E-PromptReco-v1/RECO', '20180316', 'T2_US_Florida') 22 107.86
Never read ('/TTTo2L2Nu_TuneCUETP8M1_14TeV-powheg-pythia8/PhaseIITDRFall17DR-PU200_93X_upgrade2023_realistic_v2_ext2-v1/GEN-SIM-RECO', '20171130', 'T2_DE_DESY') 42 99.48
486 ('/MinBias_TuneCUETP8M1_13TeV-pythia8/RunIISummer15GS-MCRUN2_71_V1_ext1-v1/GEN-SIM', '20180320', 'T1_US_FNAL_Disk') 18 95.22
266 ('/QCD_Flat_Pt-15to7000_TuneCUETP8M1_14TeV_pythia8/PhaseIIFall16DR82-PU200_90X_upgrade2023_realistic_v1-v4/AODSIM', '20170322', 'T1_US_FNAL_Disk') 22 91.38
181 ('/JetHT/Run2017F-v1/RAW', '20180331', 'T1_UK_RAL_Disk') 42 86.43
181 ('/JetHT/Run2017F-v1/RAW', '20180331', 'T1_RU_JINR_Disk') 42 86.43
Never read ('/MinBias_TuneCP5_inelasticON_13TeV-pythia8/RunIIFall17GS-93X_mc20

338 ('/TTTo2L2Nu_TuneCUETP8M1_14TeV-powheg-pythia8/PhaseIITDRSpring17DR-PU200_91X_upgrade2023_realistic_v3-v3/GEN-SIM-RECO', '20180629', 'T2_US_Caltech') 42 17.82
Never read ('/SingleNeutrino/RunIISummer16DR80-FlatPU0to75TuneCP5_80X_mcRun2_asymptotic_2016_TrancheIV_v6-v1/GEN-SIM-RAW', '20180627', 'T2_BR_SPRACE') 42 17.81
898 ('/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8/RunIIFall15DR76Premix-premixPU50_nondeterministic_76X_mcRun2_asymptotic_v12_ext1-v1/AODSIM', '20160417', 'T2_BR_UERJ') -99 17.81
Never read ('/AlCaPhiSym/Run2016A-v1/RAW', '20180723', 'T2_CH_CERN') 102 17.78
149 ('/SingleElectron/Run2016C-23Sep2016-v1/AOD', '20180801', 'T1_RU_JINR_Disk') 42 17.73
333 ('/JetHT/Run2016H-JetHTJetPlusHOFilter-07Aug17-v1/RAW-RECO', '20180625', 'T2_US_Caltech') 42 17.47
106 ('/DYToLL-M-50_1J_14TeV-madgraphMLM-pythia8/PhaseIITDRFall17DR-PU200_93X_upgrade2023_realistic_v2_ext1-v1/GEN-SIM-RECO', '20180621', 'T2_US_Caltech') 42 17.46
545 ('/ReggeGribovPartonMC_EposLHC_pPb_4080_4080_D